In [1]:
import os
import ast
import sqlite3
import itertools
import ipywidgets as widgets

In [2]:
# Set the base folder path for the project
output_path = "../output"
images_path = os.path.join(output_path, "images")
metadata_path = os.path.join(output_path, "metadata")
config_path = os.path.join(output_path, "config")

list_of_paths = [output_path, images_path, metadata_path, config_path]

In [3]:
dict_metadata = {}


tags = set(list(itertools.chain.from_iterable([ast.literal_eval(t) for t in dict_metadata['tags']])))
user_data = {}

# Create a label for the title
title_label = widgets.Label(value='User Information Form')

# Create text boxes for first and last name
pseudo = widgets.Text(description='Pseudo :')

# Create a color picker for favorite colors
color_picker = widgets.ColorPicker(
    concise=True,
    description='Favorite Colors:',
    value='#FF0000',
    continuous_update=False,
    disabled=False
)

# Create a dropdown list of tags
tag_dropdown = widgets.SelectMultiple(
    options=tags,
    value=[],
    description='Tags:',
    disabled=False
)

# Create a dropdown for image orientation
orientation_dropdown = widgets.Dropdown(
    options=['Portrait', 'Landscape'],
    value='Portrait',
    description='Orientation:'
)

# Create sliders for image height and width
height_slider = widgets.IntSlider(min=100, max=4000, step=100, description='Height:')
width_slider = widgets.IntSlider(min=100, max=4000, step=100, description='Width:')

#  Create a button to submit the form
submit_button = widgets.Button(description='Submit')

# Create a VBox container for the widgets
form_container = widgets.VBox([
    title_label,
    pseudo,
    color_picker,
    orientation_dropdown,
    height_slider,
    width_slider,
    tag_dropdown,
    submit_button
])

form_container.layout = widgets.Layout(
    width='600px',
    height='500px',
    justify_content='center',  # Centrer les widgets horizontalement
    align_items='center'  # Centrer les widgets verticalement
)

# Define a function to handle form submission
def on_submit_button_clicked(b):
    user_data.update({
        pseudo.value : {
            'fav_color' : color_picker.value,
            'fav_orientation' : orientation_dropdown.value,
            'fav_height' : height_slider.value,
            'fav_width' : width_slider.value,
            'tags': tag_dropdown.value
        }
    }
    )

# Attach the on_submit_button_clicked function to the button click event
clicked = submit_button.on_click(on_submit_button_clicked)

# Display the form container
display(form_container)

KeyError: 'tags'

In [ ]:
print(user_data)

In [ ]:
list_columns = ['fav_color', 'fav_orientation', 'fav_height', 'fav_width']

def save_metadata(user_data):
    try:
        # Open a connection to the database
        conn = sqlite3.connect(os.path.join(metadata_path, 'metadata.db'))
        # Create a cursor
        c = conn.cursor()
        # Create a table if it doesn't exist : filename, key, value
        c.execute(
            '''CREATE TABLE IF NOT EXISTS users (
            pseudo text PRIMARY KEY,
            fav_color text,
            fav_orientation text,
            fav_height integer,
            fav_width integer
        )''')

        nb_users = len(user_data.keys())

        check = True
        # check if all data are usable
        if nb_users > 1:
            for pseudo in user_data.keys() :
                for column in list_columns :
                    if user_data[pseudo][column] is None :
                        check = False
                        return
        else :
            check = False

        if check :
            for pseudo in user_data.keys() :
                c.execute("SELECT * FROM users WHERE pseudo=?", (pseudo,))
                if c.fetchone():
                    c.execute("UPDATE users SET fav_color=?, fav_orientation=?, fav_height=?, fav_width=? WHERE pseudo=?",
                              (user_data[pseudo]['fav_color'], user_data[pseudo]['fav_orientation'], user_data[pseudo]['fav_height'], user_data[pseudo]['fav_width'], pseudo))
                    conn.commit()
                else :
                    c.execute("INSERT INTO users VALUES (?, ?, ?, ?, ?)", (pseudo, user_data[pseudo]['fav_color'], user_data[pseudo]['fav_orientation'], user_data[pseudo]['fav_height'], user_data[pseudo]['fav_width']))
                    conn.commit()
                print(f"User {pseudo} saved to database successfully")
            conn.close()
        else :
            print("Invalid User data")

    except Exception as e:
        print("Error saving user to database : ", e)

save_metadata(user_data)
print(user_data)